In [1]:
import os

In [2]:
%pwd

'/Users/rahulshelke/Documents/Data Science/Data Science Projects/End-to-end-Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahulshelke/Documents/Data Science/Data Science Projects/End-to-end-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )

        return data_validation_config

In [9]:
import os
import pandas as pd
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [10]:
class DataValidation:

    def __init__(self, config:DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum-dataset"))
            
            for file in all_files:
                if file.split('.')[0]  not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"\nValidation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"\nValidation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e
        
    def remove_null_rows(self):
        try:
            data_path = os.path.join("artifacts", "data_ingestion", "samsum-dataset")
            all_files = os.listdir(data_path)

            for file in all_files:

                data = pd.read_csv(os.path.join(data_path, file))
                print(int(data.isna().sum().sum()), file)

                if data.isna().sum().sum() > 0:
                    with open(self.config.STATUS_FILE, "a") as f:
                        f.write(f"\nMissing in {file}: {int(data.isna().sum().sum())}")
                    data.dropna(axis=0, inplace=True)
                    data.to_csv(os.path.join(data_path, file))
                else:
                    with open(self.config.STATUS_FILE, "a") as f:
                        f.write(f"\nMissing in {file}: {int(data.isna().sum().sum())}")
        except Exception as e:
            raise e

In [11]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
    data_validation.remove_null_rows()
except Exception as e:
    raise e

0 validation.csv
0 test.csv
0 train.csv
